In [1]:
def class_sort(class_list):
	tostring_list=[]
	for now in class_list:
		tostring_list.append(str(now))
	tostring_list.sort()

	tostring_list_idx={}
	idx=0
	for nowstring in tostring_list:
		tostring_list_idx[nowstring]=idx
		idx+=1

	result=[]
	for i in range(len(class_list)):
		result.append(0)
	
	for now in class_list:
		toidx=tostring_list_idx[str(now)]
		result[toidx]=now
	return result


In [2]:
class Error(Exception):
	pass

class InvalidFormula(Error):
	def __str__(self):
		return "Invalid Formula"

class NotSupportFormula(Error):
	def __init__(self, formula):
		self.formula=formula

	def  __str__(self):
		return "Not support '%s'" % self.formula

class InvalidPlotRange(Error):
	def __str__(self):
		return "Invalid Plot Range"

class InvalidVectorInput(Error):
	def __str__(self):
		return "Invalid Vector"

class ImaginaryValue(Error):
	def __str__(self):
		return "value is imaginary part"


In [3]:
from factor import *
from term import Term
import ds
class Expression:
	def __init__(self, terms, ops):
		self.variables={}
		self.expressions=[]

		reduced=self.__reduceZero(terms, ops)
		_reduced=self.__reduceCoeff(reduced)
		self.expressions=_reduced


	def __str__(self):
		return "%s" % self.expressions


	def __repr__(self):
		return "%s" % self.expressions


	def __reduceZero(self, terms, ops):
		reduced=[]
		reduced.append(terms[0])
		for i in range(len(ops)):
			if ops[i]=='+' and len(reduced[-1].getVariables())==0 and eval(repr(reduced[-1].getAnswer()))==0:		
				# zero in reduced
				reduced.pop()
				reduced.append(terms[i+1])
				continue
			elif len(terms[i+1].getVariables())==0 and eval(repr(terms[i+1].getAnswer()))==0:						
				# zero now watching
				continue
			else:						
				# just push
				reduced.append(ops[i])
				reduced.append(terms[i+1])
		return reduced


	def __reduceCoeff(self, reduced):
		coeff_map={}
		last_operator='+'
		# x+x -> coeff_map[x]=2
		for term in reduced:
			if term not in ['+', '-']:
				coeff=term.getCoeff()
				without_coeff_factor=term.getWithoutCoeffFactor()
				without_coeff_factor_term=None
				facs=[]
				ops=[]
				if len(without_coeff_factor)!=0:
					for fac in without_coeff_factor:
						if fac not in ['*','/']:
							facs.append(fac)
						else:
							ops.append(fac)
					without_coeff_factor_term=Term(facs, ops)
				if without_coeff_factor_term==None:
					key="NONE"
				else:
					key=without_coeff_factor_term.toString()

				if coeff_map.get(key)==None:
					coeff_map[key]=[eval(last_operator+repr(coeff)),without_coeff_factor]
				else:
					coeff_map[key][0]+=eval(last_operator+repr(coeff))
			else:
				last_operator=term

		faclist=coeff_map.keys()
		faclist=ds.class_sort(faclist)
		_reduced=[]

		# coeff_map[x*y*z]=2 -> 2*x*y*z
		for fac in faclist:
			to_term=[]
			to_term_ops=[]
			coeff=coeff_map[fac][0]
			real_factor=coeff_map[fac][1]
			
			to_term.append(Number(coeff))
			if len(real_factor)>0:
				to_term_ops.append('*')
			# x*y*z 
			for each_factor in real_factor:
				if each_factor in ['*', '/']:
					to_term_ops.append(each_factor)
				else:
					to_term.append(each_factor)

			if len(_reduced)>0:
					_reduced.append('+')
	
			_reduced.append(Term(to_term, to_term_ops))
		return _reduced

		
	def getWithoutCoeffFactor(self):
		if len(self.expressions)==1:
			return self.expressions[0].getWithoutCoeffFactor()
		else:
			return False

	def getCoeff(self):
		if len(self.expressions)==1:
			return self.expressions[0].getCoeff()
		else:
			return 1
	

	def toString(self):
		string=""
		for term in self.expressions:
			if term in ['+', '-']:
				string+=term
			else:
				string+=term.toString()
		return string

	def getType(self):
		_type=self.expressions[0].getType()
		for term in self.expressions:
			if term in ['+', '-'] or _type!=term.getType():
				_type="equation"
		return _type

	def getAnswer(self):
		reduced=""
		for term in self.expressions:
			if term in ['+', '-']:
				reduced+=str(term)
			else:
				reduced+=str(term.getAnswer())
		return eval(reduced)

	def getVariables(self):
		variables=set([])
		for term in self.expressions:
			if term not in ['+', '-']:
				var=term.getVariables()
				if var!=None:
					for now in var:
						variables.add(now)
		var_list=list(variables)
		var_list.sort()
		return var_list

	def setVariable(self, variable, value):
		is_set=False
		for term in self.expressions:
			if term not in ['+', '-']:
				is_set |= term.setVariable(variable, value)
		return is_set

	def getDerivativeBy(self, by_variable):
		deri_terms=[]
		deri_ops=[]
		for term in self.expressions:
			if term in ['+', '-']:
				deri_ops.append(term)
			else:
				deri_term, deri_op = term.getDerivativeBy(by_variable)
				for each_term in deri_term:				
					deri_terms.append(each_term)
				for each_op in deri_op:
					deri_ops.append(each_op)
			
		return Expression(deri_terms, deri_ops)



ModuleNotFoundError: No module named 'factor'

In [4]:
from math import e, pi
from regex import Regex
class Factor:
	def __init__(self, param, __type):
		self.param=param
		self.__type=__type

	def setVarable(self):
		return False
	
	def getType(self):
		return self.__type
	
	def getCoeff(self):
		return 1
	
	def getWithoutCoeffFactor(self):
		return []

		
class Variable(Factor):
	def __init__(self, variable):
		Factor.__init__(self, variable, "variable")
		self.variable=variable

		self.value=None

	def __str__(self):
		return "%s" % self.variable

	def __repr__(self):
		return "%s" % self.variable

	def getCoeff(self):
		return 1

	def toString(self):
		return self.variable

	def getAnswer(self):
		return self.value

	def getVariables(self):
		var=[]
		var.append(self.variable)
		return var

	def setVariable(self, variable, value):
		if self.variable==variable:		
			self.value=float(value)
			return True
		else:
			return False

	def getDerivativeBy(self, by_variable):
		if self.variable==by_variable:	
			return Number(1)
		else:
			return Number(0)


	@staticmethod
	def isVariable(token):
		special=Regex.special()
		if special.match(token)==None:
			return True
		else:
			return False

class Number(Factor):
	def __init__(self, number):
		Factor.__init__(self, number, "number")
		self.number=number
		

	def getAnswer(self):
		return float(self.number)


	def __str__(self):
		if self.number==e:
			return "e"
		elif self.number==pi:
			return "pi"
		else:
			return "%s" % self.number

	def __repr__(self):
		if self.number==e:
			return "e"
		elif self.number==pi:
			return "pi"
		else:
			return "%s" % self.number
	
	def getCoeff(self):
		return self.getAnswer()

	def toString(self):
		return str(self.number)

	def getVariables(self):
		return list()

	def setVariable(self, variable, value):
		return False

	def getDerivativeBy(self, by_variable):
		return Number(0)

	@staticmethod
	def isNumber(token):
		e  =Regex.e()
		pi =Regex.pi()
		num=Regex.number()
		if e.match(token)==None and pi.match(token)==None and num.match(token)==None:
			return False
		else:
			return True

	@staticmethod
	def determine(token):
		rg_e  =Regex.e()
		rg_pi =Regex.pi()
		rg_num=Regex.number()
		if rg_e.match(token)!=None:
			return Number(e)
		elif rg_pi.match(token)!=None:
			return Number(pi)
		else:
			return Number(token)



class Paranthesis(Factor):
	def __init__(self, expression):
		Factor.__init__(self, expression, "paranthesis")
		self.expression=expression

	def getAnswer(self):
		return self.expression.getAnswer()
	
	def __str__(self):
		return "(%s)" % self.expression

	def __repr__(self):
		return "(%s)" % self.expression

	def toString(self):
		return '('+self.expression.toString()+')'

	def getVariables(self):
		return self.expression.getVariables()

	def setVariable(self, variable, value):
		return self.expression.setVariable(variable, value)

	def getDerivativeBy(self, by_variable):
		return self.expression.getDerivativeBy(by_variable)

	def getWithoutCoeffFactor(self):
		return self.expression

	def getCoeff(self):
		return 1



ModuleNotFoundError: No module named 'regex'

In [5]:
from tokenizer import Tokenizer
from parser import Parser
from vector import Vector
from expression import Expression
from term import Term
from factor import Number, Paranthesis
from exception import InvalidPlotRange, InvalidVectorInput, ImaginaryValue
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as m
import sympy as sp
import re
import math


class Formula:
	def __init__(self, expression):
		if type(expression)==str or type(expression)==unicode:
			expression=str(expression)
			
			tknr=Tokenizer()
			tokens=tknr.tokenize(expression)

			p=Parser()
			expr=p.parse(tokens)

			expression=expr
		
		self.expression=expression
		self.variables=self.expression.getVariables()
		self.variables.sort()

		self.setted_variable=dict((var, None) for var in self.variables)

	def __str__(self):
		return "tree : %s\nvariable : %s" % (self.getTree(), self.getVariables())

	def toString(self):
		return self.expression.toString()
	
	def getTree(self):
		return self.expression

	def getVariables(self):
		return list(self.variables)
	
	def getSettedVariables(self):
		return self.setted_variable

	def getAnswer(self):
		for variable in self.variables:
			if self.__isInitVariable(variable)==False:
				return "error - not initialize variable"
		return self.expression.getAnswer()

	def setVariable(self, variable, value):
		if self.expression.setVariable(variable, value)==True:	
			self.setted_variable[variable]=value
			return True
		else:
			return False

	def __isInitVariable(self,variable):
		if self.setted_variable.get(variable)==None:
			return False
		else:
			return True

	def getDerivativeBy(self, by_variable):
		deri_expression=self.expression.getDerivativeBy(by_variable)
		return Formula(deri_expression)

	def isContinuous(self):
		for variable in self.variables:
			if self.__isInitVariable(variable)==False:
				return "error - not initialize variable"

		mid=self.expression.getAnswer()
		alpha=float(0.00000001)
		tolerance=float(0.0000001)

		result=True
		for variable in self.variables:
			self.expression.setVariable(variable, self.setted_variable[variable]-alpha)
			left=self.expression.getAnswer()
			self.expression.setVariable(variable, self.setted_variable[variable]+alpha)
			right=self.expression.getAnswer()

			result&=(abs(left-mid)<tolerance and abs(mid-right)<tolerance)

			self.expression.setVariable(variable, self.setted_variable[variable])
		return result

					

	def isDerivativable(self):
		if self.isContinuous()!=True:
			return False

		result=True
		for variable in self.variables:
			derivate=self.expression.getDerivativeBy(variable)
			derivate.setVariable(variable, self.setted_variable[variable])
			try:
				val=derivate.getAnswer()
				print val
				result=True
			except:
				result=False
		return result
			


	def getGradient(self):
		vec_list=[]
		for variable in self.variables:
			vec_list.append(self.expression.getDerivativeBy(variable))
		return Vector(vec_list)
	
	def getDirectionalDerivative(self, vector):
		grad_vec=self.getGradient()
		unit_vec=vector.getUnitVector()

		if grad_vec.getDimension()!=unit_vec.getDimension():
			raise InvalidVectorInput()
		
		terms=[]
		terms_ops=[]
		for i in range(grad_vec.getDimension()):
			factors=[]
			factors_ops=[]
		
			expression=grad_vec.getScala(i)
			factors.append(Number(unit_vec.getScala(i)))
			factors_ops.append('*')
			factors.append(Paranthesis(expression))
			terms.append(Term(factors, factors_ops))

		for i in range(len(terms)-1):
			terms_ops.append('+')
		return Formula(Expression(terms, terms_ops))

	def getPlotImage(self, start, end, file_name):

		var_cnt=len(self.variables)
		xs=[]
		ys=[]
		max_y=1e-20
		min_y=1e20

	#	max_y_limit=1000
	#	min_y_limit=-1000
		if var_cnt>1:
			return "error - it's not one variable function"
		for x in np.arange(start, end, math.pi/2000):
			if var_cnt==1 and self.setVariable(self.variables[0], x)==False:
				return "error"
			try:
				saved_setting=self.setted_variable
				variables=self.variables
				if len(variables)>0:
					variable=variables[0]
				else:
					variable="constant"
				self.setVariable(variable, x)

				is_continue=self.isContinuous()
				if is_continue==True:
					y=self.getAnswer()
					max_y=max(max_y, y)
					min_y=min(min_y, y)
				else:
					y=np.nan
				xs.append(x)
				ys.append(y)
			except ImaginaryValue:
				pass
	#	maxy=max_y
	#	miny=min_y


		
		plt.plot(xs, ys)
	#	if max_y>max_y_limit:
	#		max_y=max_y_limit
	#	if min_y<min_y_limit:
	#		min_y=min_y_limit
	#	if min_y>max_y:
	#		max_y=maxy
	#		min_y=miny
		if max_y-min_y<1e-7:
			plt.ylim(min_y-10, max_y+10)
		#plt.ylim(min_y, max_y)
		plt.savefig(file_name)
		plt.close()
		return True
				
	
	def getLatexString(self):
		try:
			st=self.toString()
			re_st=re.sub(r"log\((?P<base>[^,;]+),{1}(?P<exponent>.*)\)", r"log(\g<exponent>,\g<base>)", st)
			latex_string=sp.latex(sp.sympify(re_st))
		except:
			latex_string=""
		return latex_string
		

if __name__ == "__main__":


	tker=Tokenizer()
	#tokens=tker.tokenize("pow(pow(x,2),0.5)")
	tokens=tker.tokenize("tan(x)")
	#tokens=tker.tokenize("sin(x,x)")
	#tokens=tker.tokenize("x+sin(x+cos(x))+(x+x)+z*(x)*(y)")
	#tokens=tker.tokenize("x+cos(x)")
	#tokens=tker.tokenize("x+sin(x+x)+pow(2*x,2)")
	#tokens=tker.tokenize("1--(y*(x-z))")
	#tokens=tker.tokenize("x+y+sin(x+sin(z))")
	#tokens=tker.tokenize("1+2+pow(2,x)")
	#tokens=tker.tokenize("1")
	#tokens=tker.tokenize("pow(2*x, 2)/x")
	#tokens=tker.tokenize("y+log(2, x)")
	#tokens=tker.tokenize("-(pow(x+x+4*x-x,2))")
	#tokens=tker.tokenize("-log(x+x+4*x-y, 2)")
	#tokens=tker.tokenize("-y+2*x+x")
	#tokens=tker.tokenize("x--x")
	#tokens=tker.tokenize("x*(x*y*-3+x*y)")
	#tokens=tker.tokenize("x+-cos(-2*x)-y-y-2*y")
	#tokens=tker.tokenize("log(e,e)")
	#tokens=tker.tokenize("2*x+sin(3*x+4*x)+x+x")
	#tokens=tker.tokenize("2*x+3*x+y+1/2*x")
	#tokens=tker.tokenize("2/z*x/3*y+x/y*z+z/y*x")


	
	print tokens
	p=Parser()
	expr = p.parse(tokens)

	formula=Formula(expr)
	#print formula
	#print formula.getVariables()
	print formula.setVariable("x", math.pi/2)
	print "asf", formula.isDerivativable()
	#print formula.setVariable("z", 4)
	#print formula.getAnswer()
	#formula.getPlotImage(-1.3,1.3, 'plot.png')
	#print formula.isContinuous()
	#print formula.getGradient()
	#print formula.getDirectionalDerivative(Vector(3,4))

	#deri_formula = formula.getDerivativeBy('x')
	#print deri_formula
	#print deri_formula.getSettedVariables()
	#print deri_formula.setVariable("x", 2)
	#print deri_formula.setVariable("y", 3)
	#print deri_formula.setVariable("z", 4)
	#print deri_formula.getAnswer()
	#deri_formula.getPlotImage(-0.3, 0.3, 'deri_plot.png')



SyntaxError: Missing parentheses in call to 'print'. Did you mean print(val)? (<ipython-input-5-f6d046c3f10e>, line 108)

In [7]:
from math import sin, cos, tan, exp, log, e
from regex import Regex
from factor import *
from term import Term
from expression import Expression
from exception import NotSupportFormula, ImaginaryValue
import numpy as np

class Function(Factor):
	def __init__(self, name=None, param=None, base=None, exponential=None):
		Factor.__init__(self, param, "function")
		self.name=name
		self.param=param
		self.base=base
		self.exponential=exponential
		
		if base!=None and exponential!=None:
			if name=='pow' and len(base.getVariables())>0 and len(exponential.getVariables())>0:
				raise NotSupportFormula(self.name+"("+base.toString()+','+exponential.toString()+")")
			elif name=='log' and len(base.getVariables())>0:
				raise NotSupportFormula(self.name+"("+base.toString()+','+exponential.toString()+")")
				
		

	def __str__(self):
		if self.param!=None:
			return "%s(%s)" % (self.name, self.param)
		else:
			return "%s(%s,%s)" % (self.name, self.base, self.exponential)

	def __repr__(self):
		if self.param!=None:
			return "%s(%s)" % (self.name, self.param)
		else:
			return "%s(%s,%s)" % (self.name, self.base, self.exponential)

	def toString(self):
		if self.param!=None:
			return self.name+'('+self.param.toString()+')'
		else:
			return self.name+'('+self.base.toString()+','+self.exponential.toString()+')'

	def getVariables(self):
		#print self.base, self.exponential
		if self.param!=None:
			return self.param.getVariables()
		elif len(self.base.getVariables())>len(self.exponential.getVariables()):
			return self.base.getVariables()
		else:
			return self.exponential.getVariables()

	def setVariable(self, variable, value):
		return self.param.setVariable(variable, value)

	def getName(self):
		return self.name

	def getParam(self):
		return self.param

	def getBase(self):
		return self.base

	def getExponential(self):
		return self.exponential

	@staticmethod
	def isSingleParamFunction(name):
		for now_regex in Regex.singleParamFunctions():
			if now_regex.match(name)!=None:
				return True
		return False

	@staticmethod
	def isDoubleParamFunction(name):
		for now_regex in Regex.doubleParamFunctions():
			if now_regex.match(name)!=None:
				return True
		return False

	@staticmethod
	def determine(name, param=None, base=None, exponential=None):
		sin=Regex.sin()
		cos=Regex.cos()
		tan=Regex.tan()
		csc=Regex.csc()
		sec=Regex.sec()
		cot=Regex.cot()
		exp=Regex.exp()
		pow=Regex.pow()
		log=Regex.log()

		if sin.match(name)!=None:
			return Sin(param)
		elif cos.match(name)!=None:
			return Cos(param)
		elif tan.match(name)!=None:
			return Tan(param)
		elif csc.match(name)!=None:
			return Csc(param)
		elif sec.match(name)!=None:
			return Sec(param)
		elif cot.match(name)!=None:
			return Cot(param)
		elif exp.match(name)!=None:
			return Exp(param)
		elif pow.match(name)!=None:
			return Pow(base, exponential)
		elif log.match(name)!=None:
			return Log(base, exponential)
		else:
			return None
		


class Sin(Function):
	def __init__(self, param):
		Function.__init__(self, name="sin", param=param)

	def getAnswer(self):
		try:
			return sin(self.param.getAnswer())
		except ZeroDivisionError:
			return np.inf

	def getDerivativeBy(self, by_variable):
		factors=[]
		ops=[]
		factors.append(Cos(self.param))
		ops.append('*')
		factors.append(Paranthesis(self.param.getDerivativeBy(by_variable)))
		return Paranthesis(Expression([Term(factors,ops)], []))


class Cos(Function):
	def __init__(self, param):
		Function.__init__(self, name="cos", param=param)

	def getAnswer(self):
		try:
			return cos(self.param.getAnswer())
		except ZeroDivisionError:
			return np.inf

	def getDerivativeBy(self, by_variable):
		factors=[]
		ops=[]
		factors.append(Number(-1))
		ops.append('*')
		factors.append(Sin(self.param))
		ops.append('*')
		factors.append(Paranthesis(self.param.getDerivativeBy(by_variable)))
		aa=Paranthesis(self.param.getDerivativeBy(by_variable))
		a=Paranthesis(Expression([Term(factors,ops)], []))
		return Paranthesis(Expression([Term(factors,ops)], []))

class Tan(Function):
	def __init__(self, param):
		Function.__init__(self, name="tan", param=param)

	def getAnswer(self):
		try:
			return tan(self.param.getAnswer())
		except ZeroDivisionError:
			return np.inf

	def getDerivativeBy(self, by_variable):
		factors=[]
		ops=[]
		factors.append(Pow(Sec(self.param),Number(2)))
		ops.append('*')
		factors.append(Paranthesis(self.param.getDerivativeBy(by_variable)))
		return Paranthesis(Expression([Term(factors,ops)], []))

class Csc(Function):
	def __init__(self, param):
		Function.__init__(self, name="csc", param=param)

	def getAnswer(self):
		try:
			return 1.0/sin(self.param.getAnswer())
		except ZeroDivisionError:
			return np.inf

	def getDerivativeBy(self, by_variable):
		factors=[]
		ops=[]
		factors.append(Number(-1))
		ops.append('*')
		factors.append(Csc(self.param))
		ops.append('*')
		factors.append(Cot(self.param))
		ops.append('*')
		factors.append(Paranthesis(self.param.getDerivativeBy(by_variable)))
		return Paranthesis(Expression([Term(factors,ops)], []))

class Sec(Function):
	def __init__(self, param):
		Function.__init__(self, name="sec", param=param)

	def getAnswer(self):
		try:
			return 1.0/cos(self.param.getAnswer())
		except ZeroDivisionError:
			return np.inf

	def getDerivativeBy(self, by_variable):
		factors=[]
		ops=[]
		factors.append(Sec(self.param))
		ops.append('*')
		factors.append(Tan(self.param))
		ops.append('*')
		factors.append(Paranthesis(self.param.getDerivativeBy(by_variable)))
		return Paranthesis(Expression([Term(factors,ops)], []))

class Cot(Function):
	def __init__(self, param):
		Function.__init__(self, name="cot", param=param)

	def getAnswer(self):
		try:
			return 1.0/tan(self.param.getAnswer())
		except ZeroDivisionError:
			return np.inf

	def getDerivativeBy(self, by_variable):
		factors=[]
		ops=[]
		factors.append(Number(-1))
		ops.append('*')
		factors.append(Pow(Csc(self.param),Number(2)))
		ops.append('*')
		factors.append(Paranthesis(self.param.getDerivativeBy(by_variable)))
		return Paranthesis(Expression([Term(factors,ops)], []))

class Exp(Function):
	def __init__(self, param):
		Function.__init__(self, name="exp", param=param)

	def getAnswer(self):
		try:
			return exp(self.param.getAnswer())
		except ZeroDivisionError:
			return np.inf
	
	def getDerivativeBy(self, by_variable):
		factors=[]
		ops=[]
		factors.append(Exp(self.param))
		ops.append('*')
		factors.append(Paranthesis(self.param.getDerivativeBy(by_variable)))
		return Paranthesis(Expression([Term(factors, ops)], []))


class Log(Function):
	def __init__(self, base, exponential):
		Function.__init__(self, name="log", base=base, exponential=exponential)

	def getAnswer(self):
		try:
			if self.exponential.getAnswer()>0:
				return log(self.exponential.getAnswer(), self.base.getAnswer())
			else:
				raise ImaginaryValue()
		except ZeroDivisionError:
			return np.inf

	def setVariable(self, variable, value):
		return (self.base.setVariable(variable, value) | self.exponential.setVariable(variable, value))

	def getDerivativeBy(self, by_variable):
		factors=[]
		ops=[]
		factors.append(Number(1))
		ops.append('*')
		factors.append(Pow(self.exponential, Number(-1)))
		ops.append('/')
		factors.append(Log(Number(e), self.base))
		ops.append('*')
		factors.append(Paranthesis(self.exponential.getDerivativeBy(by_variable)))

		return Paranthesis(Expression([Term(factors, ops)], []))


class Pow(Function):
	def __init__(self, base, exponential):
		Function.__init__(self, name="pow", base=base, exponential=exponential)

	def getAnswer(self):
		try:
			base_ans=self.base.getAnswer()
			expo_ans=self.exponential.getAnswer()

			return pow(self.base.getAnswer(), self.exponential.getAnswer())
		except ZeroDivisionError:
			return np.inf
		except ValueError:
			return 0

	def setVariable(self, variable, value):
		return (self.base.setVariable(variable, value) | self.exponential.setVariable(variable, value))

	def getDerivativeBy(self, by_variable):
		factors=[]
		ops=[]
		if len(self.base.getVariables())>0 and len(self.exponential.getVariables())==0:

			factors.append(Number(self.exponential.getAnswer()))
			ops.append('*')
			factors.append(Pow(self.base, Number(self.exponential.getAnswer()-1)))
			ops.append('*')
			factors.append(Pow(self.base.getDerivativeBy(by_variable), Number(1)))

		elif len(self.base.getVariables())==0 and len(self.exponential.getVariables())>0:
			
			factors.append(Log(Number(e), Number(self.base.getAnswer())))
			ops.append('*')
			factors.append(Pow(self.base, self.exponential))
			ops.append('*')
			factors.append(Paranthesis(self.exponential.getDerivativeBy(by_variable)))
		elif len(self.base.getVariables())==0 and len(self.exponential.getVariables())==0:
			factors.append(Number(0))
		return Paranthesis(Expression([Term(factors, ops)], []))




ModuleNotFoundError: No module named 'regex'

In [8]:
from factor import Number, Variable, Paranthesis
from functions import Function
from term import Term
from expression import Expression
from exception import InvalidFormula
class Parser:
	def __init__(self):
		self.tree=[]
		self.functions=set([])

	def __str__(self):
		return "%s" % self.tree
	
	def __repr__(self):
		return "%s" % self.tree
	
	def parse(self, tokens):
		self.tree=self.takeExpression(tokens)
		return self.tree
	
	def takeExpression(self, tokens):
		terms = []
		ops = []
		try:
			terms.append(self.takeTerm(tokens))
			while tokens.isEmpty()==False and tokens.front() in ['+','-']:
				ops.append(tokens.front())
				tokens.popFront()
				terms.append(self.takeTerm(tokens))
		except IndexError:
			raise InvalidFormula()
		return Expression(terms, ops)

	def takeTerm(self, tokens):
		factors = []
		ops = []
		try:
			factors.append(self.takeFactor(tokens))
			while tokens.isEmpty()==False and tokens.front() in ['*', '/']:
				ops.append(tokens.front())
				tokens.popFront()
				factors.append(self.takeFactor(tokens))
		except IndexError:
			raise InvalidFormula()
		return Term(factors, ops)

	def takeFactor(self, tokens):
		now = tokens.front()
		if now == '(':
			try:
				tokens.popFront()
				exp = self.takeExpression(tokens)
				tokens.popFront()
			except IndexError:
				raise InvalidFormula()
			return Paranthesis(exp)
		elif Function.isSingleParamFunction(now)==True:
			try:
				self.functions.add(now)
				tokens.popFront()
				lparan=tokens.popFront()
				exp = self.takeExpression(tokens)
				rparan=tokens.popFront()
				if (lparan=='(' and rparan==')')==False:
					raise InvalidFormula()
			except IndexError:
				raise InvalidFormula()
			return Function.determine(now, param=exp)
		elif Function.isDoubleParamFunction(now)==True:
			try:
				self.functions.add(now)
				tokens.popFront()
				lparan=tokens.popFront()	
				exp1 = self.takeExpression(tokens)
				tokens.popFront()
				exp2 = self.takeExpression(tokens)
				rparan=tokens.popFront()
				if (lparan=='(' and rparan==')')==False:
					raise InvalidFormula()
			except IndexError:
				raise InvalidFormula()
			return Function.determine(now, base=exp1, exponential=exp2)
		elif Number.isNumber(now)==True:
			tokens.popFront()
			return Number.determine(now)
		elif now == '-':
			try:
				tokens.popFront()
				fac = self.takeFactor(tokens)
				term=Term([Number(-1),fac],['*'])
				expr=Expression([term],[])
			except IndexError:
				raise InvalidFormula()
			return Paranthesis(expr)
		elif Variable.isVariable(now)==True:
			tokens.popFront()
			return Variable(now)
		else:
			raise InvalidFormula()




ModuleNotFoundError: No module named 'factor'

In [9]:
import re

class Regex:
	#__special=re.compile("[`~!@#$%^&\*()\-_=\+\\|[]{};:\'\",\n.\n<>\/?]")
	__special=re.compile("[\+\-\*\/\,\.\`\~\!\@\#\$\%\^\&\(\)\_\=\?\<\>\[\]\{\}]")

	__e  =re.compile("e",re.I)
	__pi =re.compile("pi", re.I)
	__num=re.compile("-*\d*\.\d+|-*\d+", re.I)
	
	__sin=re.compile("sine?", re.I)
	__cos=re.compile("cos|cosine", re.I)
	__tan=re.compile("tan|tangent", re.I)
	__csc=re.compile("csc|cosecant", re.I)
	__sec=re.compile("sec|secant", re.I)
	__cot=re.compile("cot|cotangent", re.I)

	__exp=re.compile("exp|exponential", re.I)
	__pow=re.compile("pow|power", re.I)
	__log=re.compile("log", re.I)

	@classmethod
	def singleParamFunctions(cls):
		 func=[]
		 func.append(cls.__sin)
		 func.append(cls.__cos)
		 func.append(cls.__tan)
		 func.append(cls.__csc)
		 func.append(cls.__sec)
		 func.append(cls.__cot)
		 func.append(cls.__exp)
		 return func

	@classmethod
	def doubleParamFunctions(cls):
		 func=[]
		 func.append(cls.__pow)
		 func.append(cls.__log)
		 return func

	@classmethod
	def e(cls):
		return cls.__e

	@classmethod
	def pi(cls):
		return cls.__pi

	@classmethod
	def pi(cls):
		return cls.__pi
	
	@classmethod
	def number(cls):
		return cls.__num

	@classmethod
	def sin(cls):
		return cls.__sin

	@classmethod
	def cos(cls):
		return cls.__cos

	@classmethod
	def tan(cls):
		return cls.__tan

	@classmethod
	def csc(cls):
		return cls.__csc
	
	@classmethod
	def sec(cls):
		return cls.__sec
	
	@classmethod
	def cot(cls):
		return cls.__cot

	@classmethod
	def exp(cls):
		return cls.__exp

	@classmethod
	def pow(cls):
		return cls.__pow
	
	@classmethod
	def log(cls):
		return cls.__log

	@classmethod
	def special(cls):
		return cls.__special


In [10]:
from factor import Number, Variable
import functions as func
import ds
class Term:
	def __init__(self, factors, ops):

		self.factors=factors
		self.ops=ops

			
		for i in range(len(self.factors)):
			if self.factors[i].getType()=="function":
				if self.factors[i].getName()=="log":
					if self.factors[i].getBase().getType()=="number" and self.factors[i].getExponential().getType()=="number" and str(self.factors[i].getBase().getAnswer())==str(self.factors[i].getExponential().getAnswer()):
						self.factors[i]=Number(1)
					elif self.factors[i].getExponential().getType()=="number" and self.factors[i].getExponential().getAnswer()==1:
						self.factors[i]=Number(0)
				elif self.factors[i].getName()=="pow":
					if self.factors[i].getBase().getType()=="number" and self.factors[i].getBase().getAnswer()==0:
						self.factors[i]=Number(0)
					elif self.factors[i].getExponential().getType()=="number" and self.factors[i].getExponential().getAnswer()==0:
						self.factors[i]=Number(1)


		reduced=self.__reduceZeroOne(factors, ops)

		self.terms=[]

		_reduced, coeff=self.__reduceCoeff(reduced)
		self.terms=_reduced
		self.coeff=coeff	
						

	def __str__(self):
		return "%s" % self.terms

	def __repr__(self):
		return "%s" % self.terms


	def __reduceZeroOne(self, factors, ops):
		reduced=[]

		reduced.append(factors[0])
		if len(factors[0].getVariables())==0 and eval(repr(factors[0].getAnswer()))==0:					
			# number 0
			reduced=[Number(0)]
		else:
			for i in range(len(ops)):
				if ops[i]=='*' and len(factors[i+1].getVariables())==0 and eval(repr(factors[i+1].getAnswer()))==0:		
					# number 0
					reduced=[Number(0)]
					break
				elif ops[i]=='*' and len(reduced[-1].getVariables())==0 and eval(repr(reduced[-1].getAnswer()))==1:		
					# number 1
					reduced.pop()
					reduced.append(factors[i+1])
					continue
				elif len(factors[i+1].getVariables())==0 and eval(repr(factors[i+1].getAnswer()))==1:					
					# (aa * 1) or (aa / 1)
					continue
				else:							
					# just push
					reduced.append(ops[i])
					reduced.append(factors[i+1])
		return reduced


	def __reduceCoeff(self, reduced):
		multiply_variable_map={}
		divide_variable_map={}
		last_operator='*'
		coeff=1
		for factor in reduced:
			if factor not in ['*', '/']:
				if len(factor.getVariables())==0:	# number
					if last_operator=='*':
						coeff*=factor.getAnswer()
					else:
						coeff/=factor.getAnswer()
				else:								# variable
					if last_operator=='*':
						exponential=multiply_variable_map.get(factor.toString())
						if exponential==None:
							multiply_variable_map[factor.toString()]=[1,factor]
						else:
							multiply_variable_map[factor.toString()][0]+=1
					else:
						exponential=divide_variable_map.get(factor.toString())
						if exponential==None:
							divide_variable_map[factor.toString()]=[1,factor]
						else:
							divide_variable_map[factor.toString()][0]+=1
			else:
				last_operator=factor
		
		# abbrevi
		for mval in multiply_variable_map.keys():
			for dval in divide_variable_map.keys():
				if mval==dval:
					multiply_variable_map[mval][0]-=1
					divide_variable_map[dval][0]-=1



		# x*x -> pow(x,2)
		multiply_variable=[]
		divide_variable=[]
		for var in multiply_variable_map.keys():
			exponential=multiply_variable_map[var][0]
			real_factor=multiply_variable_map[var][1]
			if exponential>1:
				multiply_variable.append(func.Pow(Variable(var), Number(exponential)))
			elif exponential==1:
				multiply_variable.append(real_factor)
		for var in divide_variable_map.keys():
			exponential=divide_variable_map[var][0]
			real_factor=divide_variable_map[var][1]
			if exponential>1:
				divide_variable.append(func.Pow(Variable(var), Number(exponential)))
			elif exponential==1:
				divide_variable.append(real_factor)


		multiply_variable=ds.class_sort(multiply_variable)
		divide_variable=ds.class_sort(divide_variable)

		_reduced=[]
		if (len(multiply_variable)>0 and coeff==1)==False:
			_reduced.append(Number(coeff))

		self.only_factor_list=[]
		for var in multiply_variable:
			if len(_reduced)!=0:
				_reduced.append('*')
			if len(self.only_factor_list)!=0:
				self.only_factor_list.append('*')
			self.only_factor_list.append(var)
			_reduced.append(var)
		for var in divide_variable:
			if len(_reduced)!=0:
				_reduced.append('*')
			if len(self.only_factor_list)!=0:
				self.only_factor_list.append('*')
			self.only_factor_list.append(func.Pow(var, Number(-1)))		
			# for avoid ambiguity coeff
			_reduced.append(var)
		return (_reduced, coeff)
	
	def getWithoutCoeffFactor(self):
		return self.only_factor_list
	
		
	def getCoeff(self):
		if len(self.only_factor_list)==0:		# constant
			return self.coeff
		else:
			return self.terms[0].getCoeff()		# 2*x*y, x*y



	def toString(self):
		string=""
		for factor in self.terms:
			if factor in ['*', '/']:
				string+=factor
			else:
				string+=factor.toString()
		return string
	
	def getType(self):
		_type=self.terms[0].getType()
		for fac in self.terms:
			if fac in ['*', '/'] or _type!=fac.getType():
				_type="equation"
		return _type

	def getAnswer(self):
		reduced=""
		for factor in self.terms:
			if factor in ['*', '/']:
				reduced+=str(factor)
			else:
				reduced+=str(factor.getAnswer())
		return eval(reduced)

	def getVariables(self):
		variables=set([])
		for factor in self.terms:
			if factor not in ['*', '/']:
				var=factor.getVariables()
				if var!=None:
					for now in var:
						variables.add(now)
		var_list=list(variables)
		var_list.sort()
		return var_list

	def setVariable(self, variable, value):
		is_set=False
		for factor in self.terms:
			if factor not in ['*', '/']:
				is_set |= factor.setVariable(variable, value)
		return is_set

	def getDerivativeBy(self, by_variable):
		deri_terms=[]
		deri_terms_ops=[]
		for i in range(len(self.factors)):
			deri_factors=[]
			deri_factors_ops=[]
			if i==0:
				deri_factors.append(self.factors[0].getDerivativeBy(by_variable))
			else:
				deri_factors.append(self.factors[0])
				deri_terms_ops.append('+')
			for j in range(len(self.factors[1:])):
				deri_factors_ops.append('*')
				if self.ops[j]=='*':
					if j+1==i:
						deri_factors.append(self.factors[j+1].getDerivativeBy(by_variable))
					else:
						deri_factors.append(self.factors[j+1])
				elif self.ops[j]=='/':
					if j+1==i:
						deri_factors.append(func.Pow(self.factors[j+1], Number(-1)).getDerivativeBy(by_variable))
					else:
						deri_factors.append(func.Pow(self.factors[j+1], Number(-1)))
			deri_terms.append(Term(deri_factors, deri_factors_ops))
			
		return deri_terms, deri_terms_ops



ModuleNotFoundError: No module named 'factor'

In [11]:
import re

class Tokens:
	def __init__(self, tokens):
		self.arr=tokens

	def __str__(self):
		return "%s" % (self.arr)

	def __repr__(self):
		return "%s" % (self.arr)

	def popFront(self):
		return self.arr.pop(0)

	def front(self):
		return self.arr[0]

	def size(self):
		return len(self.arr)

	def isEmpty(self):
		if self.size()==0:
			return True
		else:
			return False
	

class Tokenizer:

	def __init__(self):
		self.tokens=[]

	def tokenize(self, input_string):
		input_string=input_string.replace(" ", "")
		self.tokens=re.findall("-?\d+\.\d+|-?\d+|[\+\-\*\/\(\)]|\w+|\,", input_string)
		return Tokens(self.tokens)



if __name__ == "__main__":
	tker=Tokenizer()
	tokens=tker.tokenize("xy+sin(x+cos(x))-(x+x)")
	print tokens
	tokens=tker.tokenize("2x")
	print tokens


SyntaxError: Missing parentheses in call to 'print'. Did you mean print(tokens)? (<ipython-input-11-127c0931700f>, line 44)

In [12]:
from math import sqrt
from exception import InvalidVectorInput
'''
usage:	Vector(1,2)
	Vector([1,2])
'''

class Vector:
	def __init__(self, *args):
		self.args=[]
		if type(args[0])==list:
			self.args=args[0]
		else:
			for arg in args:
				self.args.append(arg)

	def __str__(self):
		return "%s" % self.args

	def toString(self):
		vec_st=""
		for arg in self.args:
			if vec_st!="":
				vec_st+=', '
			vec_st+=arg.toString()
		return vec_st



	def getVector(self):
		return self.args

	def getDimension(self):
		return len(self.args)

	def getScala(self, index):
		return self.args[index]
	
	def getUnitVector(self):
		sq_sum=0
		unit_vector=[]
		for arg in self.args:
			if (type(arg)==int or type(arg)==float)==False:
				raise InvalidVectorInput()
			sq_sum+=pow(arg, 2)
		for i in range(len(self.args)):
			unit_vector.append(self.args[i]/sqrt(sq_sum))
		return Vector(unit_vector)
	
if __name__=="__main__":
	vec=Vector(3, 4)
	print vec
	print vec.getUnitVector()
	for i in range(vec.getDimension()):
		print vec.getScala(i)


SyntaxError: Missing parentheses in call to 'print'. Did you mean print(vec)? (<ipython-input-12-ac71972153e2>, line 52)